In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

### Get the research labs webpage

In [2]:
url_extension = 'non_oap_facpubs/index.11.html'
LINK_INDICATOR = "facpubs/"
URL = f"https://web.uri.edu/bec/access-uris-research-and-lab-facilities/"
page = requests.get(URL)

### Parse out specific data from the page content

In [3]:
soup = BeautifulSoup(page.content, "html.parser")

Extract the link from each article listing

In [9]:
main_div = soup.find(id="main")

In [10]:
print(main_div)

In [40]:
# Get the link to the page containing more information on this dissertation.
labs = main_div.find_all("p")[1:-4] # , class_="article-listing")
labs

In [42]:
lab_links = [str(l.find_all('a')[0]).split('\"')[1] if len(l.find_all('a')) > 0 else None for l in labs]
links

In [50]:
lab_names = [str(l.find_all('strong')[0])[8:-9] if len(l.find_all('strong')) > 0 else None for l in labs]
lab_names

In [51]:
print(len(lab_names))
print(len(lab_links))

In [82]:
df_dict = {
    'who': lab_names,
    'link': lab_links,
    'desc': [l.get_text() for l in labs],
    'web_desc': [None for _ in range(len(lab_links))],
    'when': ['2023' for _ in range(len(lab_links))],
    'what': [None for _ in range(len(lab_links))]
}
df = pd.DataFrame.from_dict(df_dict)

Get each linked page and extract each desired data point from that page

In [83]:
db_entries = []

for index, row in df.iterrows():
    try:
        url = row['link']
        _lab_page = requests.get(url)
        _lab_soup = BeautifulSoup(_lab_page.content, "html.parser")
        _main_div = _lab_soup.find(id="main")
        lab_page_text = _main_div.get_text()
        row['web_desc'] = lab_page_text
        
    except Exception as e:
        print(e)

    # print("\n\nWhat: ", d['what'])
    # print("\n\nWho: ", d['who'])
    # print("\n\nWhen: ", date_awarded)


In [84]:
df

In [85]:
# Create the 'what' column through concatenation.
for index, row in df.iterrows():
    row['what'] = str(row['desc']) + str(row['web_desc'])

In [88]:
df = df.drop(['web_desc', 'desc'], axis=1)
df

In [89]:
file_name = os.getenv("RL_FILE_NAME")
file_name

In [90]:
df.to_csv(file_name)